In [1]:
import pandas as pd
import sys, os, subprocess, shutil, glob, bz2
from Bio import SeqIO
import numpy as np
import json


import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
palette = list(mcolors.TABLEAU_COLORS.keys())
%matplotlib inline


# Navigate back to workbookDir in case of re-running a code block:
if not 'workbookDir' in globals():
    workbookDir = os.getcwd()
print('workbookDir: ' + workbookDir)
os.chdir(workbookDir)  # If you changed the current working dir, this will take you back to the workbook dir.

workbookDir: /home/sulab/tRNA-charge-seq/3-stats_collection


### Requirements
1. gfsa
2. hgrewsa

In [2]:
DNAcompRNA = {a: b for a, b in zip('ATGC', 'UACG')}
def anticodon2codon(anticodon):
    codon = ''.join([DNAcompRNA[b] for b in anticodon[::-1]])
    return(codon)

In [4]:
# Variables that changes from batch to batch:
MIN_SCORE = 25
#data_folder = 'data/pilot_exp'
data_folder = 'data/tRNAseq_lib1'
# project_folder = 'projects/pilot_exp'
project_folder = 'projects/tRNAseq_lib1'
tRNA_database = '../../../2-align_reads/tRNA_database/hg19_mature-tRNA.fa'

In [5]:
# Variables that should not change:
data_dir = 'stats_collection'
align_dir = 'SWalign'
umi_dir = 'UMI_trimmed'
sample_list = 'sample_list.xlsx'

In [7]:
### Read sample information ###
sample_df = pd.read_excel('../' + project_folder + '/' + sample_list)
sample_dict = {un: {'sample_name': n, 'replicate': r, 'barcode': b} for un, n, r, b in zip(sample_df['sample_name_unique'].values, sample_df['sample_name'].values, sample_df['replicate'].values, sample_df['barcode'].values)}

# Create folder for data and stats:
os.chdir('../' + data_folder)
stats_dir = '../../' + project_folder + '/stats_collection'
try:
    os.mkdir(stats_dir) # For stats
except:
    shutil.rmtree(stats_dir)
    os.mkdir(stats_dir)
# For manipulations and final data:
try:
    os.mkdir(data_dir) # For data
except:
    shutil.rmtree(data_dir)
    os.mkdir(data_dir)
os.chdir(data_dir)

In [8]:
# Read the tRNA database to find the length of each sequence:
tRNA_data = dict()
for record in SeqIO.parse(tRNA_database, "fasta"):
    tRNA_data[record.id] = dict()
    tRNA_data[record.id]['len'] = len(record.seq)
    tRNA_data[record.id]['codon'] = anticodon2codon(record.id.split('-')[2])
    tRNA_data[record.id]['anticodon'] = record.id.split('-')[2]
    tRNA_data[record.id]['amino_acid'] = record.id.split('-')[1]

In [14]:
stat_csv_fnam = 'stats_collection.csv.bz2'
agg_csv_fnam = 'stats_filtered_CC-CCA-aggregate.csv'
try:
    pass
    os.remove(stat_csv_fnam)
    os.remove(agg_csv_fnam)
except:
    pass

# Open filehandles and printer headers:
fh_stats_out = bz2.open(stat_csv_fnam, 'ab')
header = ['readID', 'sample_name', 'replicate', 'barcode', 'tRNA_annotation', 'align_score', 'unique_annotation', 'tRNA_annotation_len', 'align_5p_idx', 'align_3p_idx', 'align_5p_nt', 'align_3p_nt', 'codon', 'anticodon', 'amino_acid', '5p_cover', '3p_cover', '5p_non-temp', '3p_non-temp']
header_bin = str.encode(','.join(header))
fh_stats_out.write(header_bin)

fh_agg_out = open(agg_csv_fnam, 'a')
agg_cols = ['sample_name', 'replicate', 'barcode', 'tRNA_annotation', 'tRNA_annotation_len', 'unique_annotation', 'align_3p_nt', 'codon', 'anticodon', 'amino_acid', 'count']
print(','.join(agg_cols), file=fh_agg_out)

# Files to collect stats from:
json_files = glob.glob('../' + align_dir + '/*.json.bz2')
# print(json_files)
for json_file in json_files:
    fnam_base = '_'.join(json_file.split('/')[-1].split('_')[0:-1])
    uidx = fnam_base.index('UMI')
    unique_sample_name = fnam_base[0:uidx-1]
    print('Processing: {}'.format(unique_sample_name))
    with bz2.open(json_file, 'rt', encoding="utf-8") as fh_gz:
        SWres = json.load(fh_gz)
    
    # Extract non-template bases from UMI processed reads:
    fastq_fnam = '../' + umi_dir + '/' + fnam_base + '.fastq.bz2'
    with bz2.open(fastq_fnam, 'rt') as fh_gz:
        for UMIread in SeqIO.parse(fh_gz, "fastq"):
            if UMIread.id in SWres:
                qpos = SWres[UMIread.id]['qpos'][0]
                SWres[UMIread.id]['5p_non-temp'] = str(UMIread.seq)[0:(qpos[0]-1)]
                SWres[UMIread.id]['3p_non-temp'] = str(UMIread.seq)[qpos[1]:]

    ### Collect stats on a per sample basis and store in tmp file ###
    with open('tmp_stat.csv', 'w') as tmp_csv:
        print(','.join(header), file=tmp_csv)
        for i, readID in enumerate(SWres):
            sample_name = sample_dict[unique_sample_name]['sample_name']
            replicate = sample_dict[unique_sample_name]['replicate']
            barcode = sample_dict[unique_sample_name]['barcode']
            tRNA_annotation = SWres[readID]['name']
            tRNA_annotation_first = tRNA_annotation.split('@')[0]
            align_score = SWres[readID]['score']
            unique_annotation = '@' not in tRNA_annotation
            tRNA_annotation_len = tRNA_data[tRNA_annotation_first]['len']
            align_5p_idx, align_3p_idx = SWres[readID]['dpos'][0]
            align_5p_nt = SWres[readID]['qseq'][0]
            align_3p_nt = SWres[readID]['qseq'][-1]
            # Move index for reads with beta-eliminated A:
            if align_3p_idx == (tRNA_annotation_len - 1) and align_3p_nt == 'C':
                align_3p_idx += 1
            codon = tRNA_data[tRNA_annotation_first]['codon']
            anticodon = tRNA_data[tRNA_annotation_first]['anticodon']
            amino_acid = tRNA_data[tRNA_annotation_first]['amino_acid']
            _5p_cover = align_5p_idx == 1
            _3p_cover = align_3p_idx == tRNA_annotation_len
            _5p_non_temp = SWres[readID]['5p_non-temp']
            _3p_non_temp = SWres[readID]['3p_non-temp']
            # Print line to tmp file:
            csv_line = ','.join(map(str, [readID, sample_name, replicate, barcode, tRNA_annotation, align_score, unique_annotation, tRNA_annotation_len, align_5p_idx, align_3p_idx, align_5p_nt, align_3p_nt, codon, anticodon, amino_acid, _5p_cover, _3p_cover, _5p_non_temp, _3p_non_temp]))
            print(csv_line, file=tmp_csv)
    # Append the sample statistics to the final csv file using bz2 compression:
    with open('tmp_stat.csv', 'rb') as tmp_csv:
        fh_stats_out.write(tmp_csv.read())

    ### Aggregate filtered data to count charged/uncharged tRNAs ###
    # Read stats from tmp csv file (this is faster than building row by row):
    stat_df = pd.read_csv('tmp_stat.csv', keep_default_na=False)
    os.remove('tmp_stat.csv')
    # 3' must be covered and no 3' non-template bases:
    row_mask = (stat_df['3p_cover']) & (stat_df['3p_non-temp'] == '')
    agg_df = stat_df.loc[row_mask, agg_cols[0:-1]]
    agg_df['count'] = stat_df.loc[row_mask, ['align_3p_nt']]  # dummy for groupby count
    agg_df = agg_df.groupby(agg_cols, as_index=False).agg({"count": "count"})
    agg_df.to_csv(fh_agg_out, header=False, index=False, mode='a')

fh_stats_out.close()
fh_agg_out.close()

Mu3
HCS1
HVP2
L-2
L-1+2
HR40V1
Mu2
HR80V1
HR80S1
2U1
HR40P2
HR40P1
Fix1
HR40S1
2U2
HR80V2
HVS1
Fix2
HR30P1
L-1
BVS2
0U1
BAV2
HR20P1
Mu4
FT
Tu2
HR30P2
HR20S2
Rich-2
BVP2
HVP1
U-1+2
BVP1
Li3
U-1
4U1
U-2
HAV1
BVV1
HVV2
8U1
0U2
Rich-1+2
HVV1
HR20V1
HR30S2
Tu3
Li1
Li2
HCS2
Rich-1
BVV2
Tu1
HR30V2
Mu1
A-2
8U2
HR20P2
A-1+2


KeyboardInterrupt: 